In [1]:
import pandas as pd
import numpy as np

import warnings

warnings.filterwarnings('ignore')

In [2]:
df_org = pd.read_csv("./data/scout_missings_filled_2.csv")
df = df_org.copy()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15915 entries, 0 to 15914
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   make_model               15915 non-null  object 
 1   body_type                15915 non-null  object 
 2   price                    15915 non-null  int64  
 3   vat                      15915 non-null  object 
 4   km                       15915 non-null  float64
 5   prev_owner               15915 non-null  float64
 6   hp                       15915 non-null  float64
 7   inspection_new           15915 non-null  object 
 8   warranty                 15915 non-null  float64
 9   body_color               15915 non-null  object 
 10  paint_type               15915 non-null  object 
 11  nr_of_doors              15915 non-null  float64
 12  nr_of_seats              15915 non-null  float64
 13  gearing_type             15915 non-null  object 
 14  displacement          

## Additional Feature Engineering

In [4]:
# split make_model
df[['make', 'model']] = df['make_model'].str.split(' ', expand=True)
df.drop(columns=['make_model'], inplace=True)

# drop weight column
df.drop(columns=['weight'], inplace=True)

# lower Weight to weight
df.columns = df.columns.str.lower()

# lower and underscore spacings
df['body_type'] = df['body_type'].str.lower().str.replace(' ', '_').str.replace('-', '_')
df['vat'] = df['vat'].str.lower()
df['inspection_new'] = df['inspection_new'].str.lower().str.replace(' ', '_')
df['body_color'] = df['body_color'].str.lower()
df['paint_type'] = df['paint_type'].str.lower().str.replace(' ', '_').str.replace('/', '_')
df['gearing_type'] = df['gearing_type'].str.lower().str.replace('-', '_')
df['drive_chain'] = df['drive_chain'].str.lower()
df['emission_class'] = df['emission_class'].str.lower().str.replace(' ', '_')
df['new_used'] = df['new_used'].str.lower().str.replace(' ', '_').str.replace('-', '_').str.replace("'", '')
df['upholstery_material'] = df['upholstery_material'].str.lower().str.replace(' ', '_')

# fill nan
df['fuel_type'].fillna('others',inplace=True)
df['upholstery_material'].fillna('other',inplace=True)
median_value_city = df['consumption_city'].median()
df['consumption_city'] = df['consumption_city'].fillna(median_value_city)
median_value_country = df['consumption_country'].median()
df['consumption_country'] = df['consumption_country'].fillna(median_value_country)
df['upholstery_material'] = df['upholstery_material'].fillna("'none_specified'")
df['comfort_convenience'] = df['comfort_convenience'].fillna("'none_specified'")
df['entertainment_media'] = df['entertainment_media'].fillna("'none_specified'")
df['extras'] = df['extras'].fillna("'none_specified'")
df['safety_security'] = df['safety_security'].fillna("'none_specified'")

## Change object Dtypes to Categorical

In [5]:
# Select columns with dtype 'object' and convert them to categorical
df[df.select_dtypes(include='object').columns] = df.select_dtypes(include='object').apply(lambda x: x.astype('category'))

# convert to categories
df['prev_owner'] = df['prev_owner'].astype('category')
df['nr_of_doors'] = df['nr_of_doors'].astype('category')
df['nr_of_seats'] = df['nr_of_seats'].astype('category')
df['cylinders'] = df['cylinders'].astype('category')
df['gears'] = df['gears'].astype('category')
df['electricity_consumption'] = df['electricity_consumption'].astype('category')

# conver to string
df['comfort_convenience'] = df['comfort_convenience'].astype('string')
df['entertainment_media'] = df['entertainment_media'].astype('string')
df['extras'] = df['extras'].astype('string')
df['safety_security'] = df['safety_security'].astype('string')

In [6]:
# get unique categorical values for each column
unique = df.nunique()
dtypes = df.dtypes
col_info = pd.DataFrame({'unique_values': unique,
                         'dtype': dtypes})

col_info

,unique_values,dtype
body_type,9,category
price,2952,int64
vat,3,category
km,6693,float64
prev_owner,5,category
hp,80,float64
inspection_new,2,category
warranty,42,float64
body_color,13,category
paint_type,3,category


## One Hot Encoding using pd.get_dummies()

In [7]:
cat_cols = list(col_info[(col_info.dtype=='category')].index)
cat_cols

['body_type',
 'vat',
 'prev_owner',
 'inspection_new',
 'body_color',
 'paint_type',
 'nr_of_doors',
 'nr_of_seats',
 'gearing_type',
 'cylinders',
 'drive_chain',
 'emission_class',
 'gears',
 'electricity_consumption',
 'new_used',
 'fuel_type',
 'upholstery_material',
 'make',
 'model']

In [8]:
df_dummies_encoded = pd.get_dummies(df, columns=cat_cols)
df_dummies_encoded.sample(50)

,price,km,hp,warranty,displacement,co2_emission,comfort_convenience,entertainment_media,extras,safety_security,...,make_Renault,model_A1,model_A2,model_A3,model_Astra,model_Clio,model_Corsa,model_Duster,model_Espace,model_Insignia
7512,14300,19327.0,81.0,24.0,1598.0,97.0,'none_specified','none_specified','none_specified','none_specified',...,False,False,False,False,True,False,False,False,False,False
14536,10750,6800.0,66.0,0.0,898.0,113.0,"'Air conditioning', 'Cruise control', 'Electri...","'Bluetooth', 'Digital radio', 'On-board comput...",'Voice Control',"'ABS', 'Central door lock with remote control'...",...,True,False,False,False,False,True,False,False,False,False
6471,18450,30725.0,76.0,0.0,999.0,104.0,"'Air conditioning', 'Armrest', 'Cruise control...","'Digital radio', 'On-board computer', 'Radio'",'Alloy wheels',"'ABS', 'Central door lock', 'Daytime running l...",...,False,False,False,False,True,False,False,False,False,False
9406,8950,72000.0,66.0,12.0,1398.0,120.0,"'Air conditioning', 'Cruise control', 'Hill Ho...","'Bluetooth', 'Radio'",'none_specified',"'ABS', 'Central door lock', 'Daytime running l...",...,False,False,False,False,False,False,True,False,False,False
3009,14999,80781.0,81.0,0.0,1598.0,105.0,"'Air conditioning', 'Armrest', 'Automatic clim...","'Bluetooth', 'CD player', 'Hands-free equipmen...",'Alloy wheels',"'ABS', 'Central door lock', 'Daytime running l...",...,False,False,False,True,False,False,False,False,False,False
15853,43990,13.0,147.0,48.0,1997.0,139.0,"'Air conditioning', 'Armrest', 'Automatic clim...","'Bluetooth', 'Digital radio', 'Hands-free equi...","'Alloy wheels', 'Touch screen', 'Voice Control'","'ABS', 'Central door lock', 'Daytime running l...",...,True,False,False,False,False,False,False,False,True,False
412,15450,34011.0,85.0,12.0,1598.0,97.0,"'Air conditioning', 'Armrest', 'Automatic clim...","'Bluetooth', 'Hands-free equipment', 'On-board...","'Alloy wheels', 'Shift paddles'",'Isofix',...,False,True,False,False,False,False,False,False,False,False
4948,20400,17000.0,85.0,0.0,1598.0,106.0,"'Air conditioning', 'Armrest', 'Automatic clim...","'Bluetooth', 'CD player', 'MP3', 'On-board com...","'Alloy wheels', 'Voice Control'","'ABS', 'Central door lock', 'Daytime running l...",...,False,False,False,True,False,False,False,False,False,False
6781,12990,53951.0,74.0,0.0,1399.0,128.0,"'Air conditioning', 'Cruise control', 'Electri...","'Bluetooth', 'Hands-free equipment', 'On-board...","'Catalytic Converter', 'Voice Control'","'ABS', 'Central door lock', 'Daytime running l...",...,False,False,False,False,True,False,False,False,False,False
15615,28650,15.0,118.0,0.0,1997.0,135.0,"'Air conditioning', 'Cruise control', 'Park Di...",'On-board computer','Alloy wheels',"'ABS', 'Central door lock', 'Driver-side airba...",...,True,False,False,False,False,False,False,False,True,False


In [9]:
df_dummies_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15915 entries, 0 to 15914
Columns: 117 entries, price to model_Insignia
dtypes: bool(102), float64(10), int64(1), string(4)
memory usage: 3.4 MB


## Using sklearn.preprocessing.OneHotEncoder

In [10]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Apply OneHotEncoder to each column and concatenate
encoded_df_list = []
for column in cat_cols:
    encoded = encoder.fit_transform(df[[column]])
    encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out([column]))
    encoded_df_list.append(encoded_df)

df_sk_encoded = pd.concat(encoded_df_list, axis=1)

df_sk_encoded.sample(50)

,body_type_compact,body_type_convertible,body_type_coupe,body_type_off_road,body_type_other,body_type_sedans,body_type_station_wagon,body_type_transporter,body_type_van,vat_price negotiable,...,make_Renault,model_A1,model_A2,model_A3,model_Astra,model_Clio,model_Corsa,model_Duster,model_Espace,model_Insignia
3293,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
11436,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3290,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5538,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9085,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9912,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2291,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12612,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4715,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
766,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
df_sk_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15915 entries, 0 to 15914
Columns: 102 entries, body_type_compact to model_Insignia
dtypes: float64(102)
memory usage: 12.4 MB
